# Compile boostvqe circuit
This notebook serves to run boostvqe and extract the transpiled circuit.

In [1]:
import numpy as np
import qibo
from qibo import hamiltonians
from qibo.backends import construct_backend
from qibo.quantum_info.metrics import fidelity
from boostvqe.ansatze import VQE, build_circuit
from boostvqe.models.dbi import double_bracket_evolution_oracles
from boostvqe.models.dbi.double_bracket_evolution_oracles import (
    FrameShiftedEvolutionOracle,
    IsingNNEvolutionOracle,
    MagneticFieldEvolutionOracle,
    XXZ_EvolutionOracle,
)
from boostvqe.models.dbi.group_commutator_iteration_transpiler import (
    DoubleBracketRotationType,
    GroupCommutatorIterationWithEvolutionOracles,
)
from boostvqe.utils import (
    OPTIMIZATION_FILE,
    PARAMS_FILE,
    build_circuit,
    optimize_D,
    select_recursion_step_gd_circuit,
)
import json
import time
import matplotlib.pyplot as plt
from ionq_utils import *

In [2]:
def report(vqe, hamiltonian, gci, step, eo_d, mode):
    energies = hamiltonian.eigenvalues()
    ground_state_energy = float(energies[0])
    vqe_energy = float(hamiltonian.expectation(vqe.circuit().state()))
    gci_loss = float(gci.loss(step, eo_d, mode))
    gap = float(energies[1] - energies[0] + 1e-6)

    return (
        dict(
            nqubits=hamiltonian.nqubits,
            gci_loss=float(gci_loss),
            vqe_energy=float(vqe_energy),
            target_energy=ground_state_energy,
            diff_vqe_target=vqe_energy - ground_state_energy,
            diff_gci_target=gci_loss - ground_state_energy,
            gap=gap,
            diff_vqe_target_perc=abs(vqe_energy - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            diff_gci_target_perc=abs(gci_loss - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            fidelity_witness_vqe=1 - (vqe_energy - ground_state_energy) / gap,
            fidelity_witness_gci=1 - (gci_loss - ground_state_energy) / gap,
            fidelity_vqe=fidelity(vqe.circuit().state(), hamiltonian.ground_state()),
            fidelity_gci=fidelity(
                gci.get_composed_circuit()().state(), hamiltonian.ground_state()
            ),
        )
        | gci.get_gate_count_dict()
    )

def print_report(report: dict):
    print(
        f"\
    The target energy is {report['target_energy']}\n\
    The VQE energy is {report['vqe_energy']} \n\
    The DBQA energy is {report['gci_loss']}. \n\
    The difference is for VQE is {report['diff_vqe_target']} \n\
    and for the DBQA {report['diff_gci_target']} \n\
    which can be compared to the spectral gap {report['gap']}.\n\
    The relative difference is \n\
        - for VQE {report['diff_vqe_target_perc']}% \n\
        - for DBQA {report['diff_gci_target_perc']}%.\n\
    The energetic fidelity witness of the ground state is: \n\
        - for the VQE  {report['fidelity_witness_vqe']} \n\
        - for DBQA {report['fidelity_witness_gci']}\n\
    The true fidelity is \n\
        - for the VQE  {report['fidelity_vqe']}\n\
        - for DBQA {report['fidelity_gci']}\n\
                    "
    )
    print(
        f"The boosting circuit used {report['nmb_cnot']} CNOT gates coming from compiled XXZ evolution and {report['nmb_cz']} CZ gates from VQE.\n\
For {report['nqubits']} qubits this gives n_CNOT/n_qubits = {report['nmb_cnot_relative']} and n_CZ/n_qubits = {report['nmb_cz_relative']}"
    )

In [3]:
qibo.set_backend("numpy")
vqe_backend = construct_backend(backend="numpy")

[Qibo 0.2.11|INFO|2024-08-29 19:11:54]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0


In [4]:
nqubits = 5
nlayers = 3
epochs = 100
path = "./results/big_architectures_small_lr/sgd_5q_3l_42/"
OPTIMIZATION_METHOD="sgd"
OPTIMIZATION_CONFIG="{ \"maxiter\": 2, \"gd_epochs\":2}"

In [5]:
# load data
opt_options = json.loads(OPTIMIZATION_CONFIG)

params = np.array(
    np.load(path + PARAMS_FILE, allow_pickle=True).tolist()[0][epochs]
)

In [6]:
hamiltonian = hamiltonians.XXZ(nqubits, delta=0.5, backend=vqe_backend)

In [7]:
vqe = VQE(
    build_circuit(
        nqubits=nqubits,
        nlayers=nlayers,
    ),
    hamiltonian=hamiltonian,
)
vqe.circuit.set_parameters(params)

In [8]:
print(vqe.circuit.draw())

q0: ─RY─RZ─o───RY─RZ───o─RY─RZ─o───RY─RZ───o─RY─RZ─o───RY─RZ───o─RY─
q1: ─RY─RZ─Z───RY─RZ─o─|─RY─RZ─Z───RY─RZ─o─|─RY─RZ─Z───RY─RZ─o─|─RY─
q2: ─RY─RZ───o─RY─RZ─Z─|─RY─RZ───o─RY─RZ─Z─|─RY─RZ───o─RY─RZ─Z─|─RY─
q3: ─RY─RZ───Z─RY─RZ───|─RY─RZ───Z─RY─RZ───|─RY─RZ───Z─RY─RZ───|─RY─
q4: ─RY─RZ─────RY─RZ───Z─RY─RZ─────RY─RZ───Z─RY─RZ─────RY─RZ───Z─RY─


In [15]:
base_oracle = XXZ_EvolutionOracle.from_nqubits(
    nqubits=nqubits, delta=0.5, steps=2, order=2
)
# steps: DBI steps
# order: Trotter order

oracle = FrameShiftedEvolutionOracle.from_evolution_oracle(
    before_circuit=vqe.circuit.invert(),
    after_circuit=vqe.circuit,
    base_evolution_oracle=base_oracle,
)
db_rotation = DoubleBracketRotationType.group_commutator_third_order_reduced
gci = GroupCommutatorIterationWithEvolutionOracles(
        oracle,
        db_rotation
    )

In [16]:
eo_d_type = getattr(double_bracket_evolution_oracles, "IsingNNEvolutionOracle")
print(
        f"The gci mode is {gci.double_bracket_rotation_type} rotation with {eo_d_type.__name__} as the oracle.\n"
    )

The gci mode is DoubleBracketRotationType.group_commutator_third_order_reduced rotation with IsingNNEvolutionOracle as the oracle.



In [17]:
NSTEPS = 1
for gci_step_nmb in range(NSTEPS):
    print(f"Optimizing GCI step {gci_step_nmb+1} with optimizer {OPTIMIZATION_METHOD}")
    it = time.time()
    if OPTIMIZATION_METHOD == "sgd":
            params = (
                [4 - np.sin(x / 3) for x in range(nqubits)]
                if eo_d_type == MagneticFieldEvolutionOracle
                else [4 - np.sin(x / 3) for x in range(nqubits)] + nqubits * [1]
            )
            mode, best_s, best_b, eo_d = select_recursion_step_gd_circuit(
                gci,
                mode=db_rotation,
                eo_d_type=eo_d_type,
                params=params,
                step_grid=np.linspace(1e-5, 2e-2, 30),
                lr_range=(1e-3, 1),
                nmb_gd_epochs=opt_options["gd_epochs"],
                threshold=1e-4,
                max_eval_gd=30,
            )

            opt_dict = {"sgd_extras": "To be defined"}
    else:
        if gci_step_nmb == 0:
            p0 = [0.01]
            if eo_d_type == MagneticFieldEvolutionOracle:
                p0.extend([4 - np.sin(x / 3) for x in range(nqubits)])
            elif eo_d_type == IsingNNEvolutionOracle:
                p0.extend(
                    [4 - np.sin(x / 3) for x in range(nqubits)] + nqubits * [1]
                )

        else:
            p0 = [best_s]
            p0.extend(best_b)
        optimized_params, opt_dict = optimize_D(
            params=p0,
            gci=gci,
            eo_d_type=eo_d_type,
            mode=db_rotation,
            method=OPTIMIZATION_METHOD,
            **opt_options,
        )
        best_s = optimized_params[0]
        best_b = optimized_params[1:]
        eo_d = eo_d_type.load(best_b)
    print(hamiltonian.eigenvalues())
    print(f"Total optimization time required: {time.time() - it} seconds")
    gci.mode_double_bracket_rotation = db_rotation

    gci(best_s, eo_d, db_rotation)
    this_report = report(vqe, hamiltonian, gci, best_s, eo_d, db_rotation)
    print_report(this_report)



Optimizing GCI step 1 with optimizer sgd
Just finished the selection: better loss -6.157919560767338 for mode DoubleBracketRotationType.group_commutator_third_order_reduced,                with duration s=0.0150025, and eo_d name = IsingNNEvolutionOracle
[-6.28051377 -6.28051377 -6.28051377 -6.28051377 -2.73606798 -2.73606798
 -2.73606798 -2.73606798 -2.62310563 -2.62310563 -1.17570558 -1.17570558
 -1.17570558 -1.17570558  1.41177356  1.41177356  1.41177356  1.41177356
  1.73606798  1.73606798  1.73606798  1.73606798  2.04444579  2.04444579
  2.04444579  2.04444579  2.5         2.5         4.5         4.5
  5.62310563  5.62310563]
Total optimization time required: 2.731653928756714 seconds
    The target energy is -6.280513769031028
    The VQE energy is -5.605158270201547 
    The DBQA energy is -6.244022270908618. 
    The difference is for VQE is 0.6753554988294814 
    and for the DBQA 0.036491498122409816 
    which can be compared to the spectral gap 1e-06.
    The relative diffe

In [18]:
c = gci.get_composed_circuit(best_s, eo_d)
print(c.draw())

q0:     ─o────o──────────────────────X─RZ─X─RZ─o────o──────────────────────X─R ...
q1:     ─X─RZ─X─o────o───────────────|────|─RZ─X─RZ─X─o────o───────────────|── ...
q2:     ────────X─RZ─X─o────o────────|────|─RZ────────X─RZ─X─o────o────────|── ...
q3:     ───────────────X─RZ─X─o────o─|────|─RZ───────────────X─RZ─X─o────o─|── ...
q4:     ──────────────────────X─RZ─X─o────o─RZ──────────────────────X─RZ─X─o── ...

q0: ... Z─X─RZ─RY─RZ─o───RY─RZ───o─RY─RZ─o───RY─RZ───o─RY─RZ─o───RY─RZ───o─RY─ ...
q1: ... ──|─RZ─RY─RZ─Z───RY─RZ─o─|─RY─RZ─Z───RY─RZ─o─|─RY─RZ─Z───RY─RZ─o─|─RY─ ...
q2: ... ──|─RZ─RY─RZ───o─RY─RZ─Z─|─RY─RZ───o─RY─RZ─Z─|─RY─RZ───o─RY─RZ─Z─|─RY─ ...
q3: ... ──|─RZ─RY─RZ───Z─RY─RZ───|─RY─RZ───Z─RY─RZ───|─RY─RZ───Z─RY─RZ───|─RY─ ...
q4: ... ──o─RZ─RY─RZ─────RY─RZ───Z─RY─RZ─────RY─RZ───Z─RY─RZ─────RY─RZ───Z─RY─ ...

q0: ... ─────────────────────RZ─o─RY─X─RY─o────X───RZ─o──────X───RZ─────────── ...
q1: ... ───X───RZ─o──────X───RZ─|────|────|─RZ─o───RY─X───RY─o─────────X───RZ─ ...
q2

## Transform into Qiskit circuit

In [19]:
from qibo import gates, models
import qiskit.qasm2
def qibo_to_qiskit_circuit(circuit):
    qasm_code = models.Circuit.to_qasm(circuit)
    return qiskit.qasm2.loads(qasm_code)

In [20]:
vqe_c = qibo_to_qiskit_circuit(vqe.circuit)
# vqe_c.draw()

In [21]:
from qibo import gates, models
import qiskit.qasm2
gci_c = qibo_to_qiskit_circuit(c)
# gci_c.draw()

In [25]:
from qiskit.quantum_info import Statevector
state_vec_vqe = Statevector.from_instruction(vqe_c)
expectation_value_vqe = np.real(np.dot(state_vec_vqe.data.conj().T, np.dot(hamiltonian.matrix, state_vec_vqe.data)))
print("VQE energy:", expectation_value_vqe)

VQE energy: -5.605158270201548


In [26]:
state_vec_gci = Statevector.from_instruction(gci_c)
expectation_value_gci = np.real(np.dot(state_vec_gci.data.conj().T, np.dot(hamiltonian.matrix, state_vec_gci.data)))
print("GCI energy:", expectation_value_gci)

GCI energy: -6.244022270908637


In [28]:
# contribution by components
xx = xxz_hamiltonian(nqubits,delta=1,select='XX')
yy = xxz_hamiltonian(nqubits,delta=1,select='YY')
zz = xxz_hamiltonian(nqubits,delta=1,select='ZZ')
coefs = [1,1,0.5]
observables = [xx,yy,zz]
for i, ob in enumerate(observables):
    expectation = coefs[i]*np.real(np.dot(state_vec_gci.data.conj().T, np.dot(ob.to_matrix(), state_vec_gci.data)))
    print(expectation)

-2.5613351599173666
-2.56083299909141
-1.1218541118998608


In [ ]:
# expectations from Qibo
energies = hamiltonian.eigenvalues()
ground_state_energy = float(energies[0])
print('Target energy: ', ground_state_energy)
vqe_energy = float(hamiltonian.expectation(vqe.circuit().state()))
print("VQE energy:", vqe_energy)
gci_loss = float(gci.loss(best_s, eo_d, db_rotation))
print("DBQA energy:", gci_loss)

Target energy:  -6.280513769031028
VQE energy: -5.605158270201547
DBQA energy: -6.24402227092173


# Run on IonQ simulator

In [29]:
from qiskit import QuantumCircuit
from qiskit_ionq import IonQProvider
from copy import deepcopy

my_api_key = "pOiUVlzriOoF2wX1kp3lIqid1OMhwXZ5"
provider = IonQProvider(my_api_key)
simulator_backend = provider.get_backend("ionq_simulator")

In [30]:
# generate rotated gci circuits for measurement in different bases
qc_XYZ = rotate_circuit_XYZ(gci_c)

In [31]:
shots = 1000
keys_ls = []
frequencies_ls = []
observable_labels = ['XX', 'YY', 'ZZ']
xxz_expectation = 0
for i,qc_m in enumerate(qc_XYZ):
    job = simulator_backend.run(qc_m, shots=shots)
    result = job.result()
    # Get the counts from the result
    counts = result.get_counts()
    keys = [key.split()[0][::-1] for key in counts.keys()]
    sample_counts = list(counts.values())
    frequencies = np.array(sample_counts) / sum(sample_counts)
    ob = zz
    coef = coefs[i]
    expectation = sample_to_expectation(ob.to_matrix(), keys, frequencies)*coef
    xxz_expectation += expectation
    print(observable_labels[i], expectation)
    keys_ls.append(keys)
    frequencies_ls.append(frequencies)
    
print('Expectation energy:', xxz_expectation)

XX -2.544000000000001
YY -2.588000000000001
ZZ -1.1620000000000001
Expectation energy: -6.294000000000001
